In [1]:
import tensorflow as tf
print(tf.__version__)

2.17.0


In [2]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Scale the inputs in range of (-1, +1) for better training
x_train, x_test = x_train / 255.0 * 2 - 1, x_test / 255.0 * 2 - 1

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Flatten the data
N, H, W = x_train.shape
D = H * W
x_train = x_train.reshape(-1, D)
x_test = x_test.reshape(-1, D)

In [5]:
latent_dim = 100

In [6]:
# Defining the generator model
def build_generator(latent_dim):
  i = Input(shape=(latent_dim,))
  x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
  x = BatchNormalization(momentum=0.7)(x)
  x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
  x = BatchNormalization(momentum=0.7)(x)
  x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
  x = BatchNormalization(momentum=0.7)(x)
  x = Dense(D, activation='tanh')(x)

  model = Model(i, x)
  return model

In [7]:
# Defining the discriminator model
def build_discriminator(img_size):
  i = Input(shape=(img_size,))
  x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
  x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
  x = Dense(1, activation='sigmoid')(x)


  model = Model(i, x)
  return model

In [8]:
# Build and compile the discriminator
discriminator = build_discriminator(D)
discriminator.compile ( loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build and compile the combined model
generator = build_generator(latent_dim)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [9]:
# Create an input to represent noise sample from latent space
z = Input(shape=(latent_dim,))

z.shape

(None, 100)

In [10]:
img = generator(z)

In [11]:
discriminator.trainable = False

In [12]:
fake_pred = discriminator(img)

In [13]:
# Create the combined model object
combined_model_gen = Model(z, fake_pred)

In [14]:
combined_model_gen.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))


In [15]:
batch_size = 32
epochs = 30000
sample_period = 200 # every `sample_period` steps generate and save some data"

In [16]:
# Create batch labels to use when calling train_on_batch
ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

# Store the losses
d_losses = []
g_losses = []

# Create a folder to store generated images
if not os.path.exists('gan_images'):
  os.makedirs('gan_images')

In [17]:
def sample_images(epoch):
  rows, cols = 5, 5
  noise = np.random.randn(rows * cols, latent_dim)
  imgs = generator.predict(noise)

  # Rescale images 0 - 1
  imgs = 0.5 * imgs + 0.5

  fig, axs = plt.subplots(rows, cols)
  idx = 0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx].reshape(H, W), cmap='gray')
      axs[i,j].axis('off')
      idx += 1
  fig.savefig("gan_images/%d.png" % epoch)
  plt.close()

In [18]:
# Main training loop
for epoch in range(epochs):
  ###########################
  ### Train discriminator ###
  ###########################

  # Select a random batch of images
  idx = np.random.randint(0, x_train.shape[0], batch_size)
  real_imgs = x_train[idx]

  # Generate fake images
  noise = np.random.randn(batch_size, latent_dim)
  fake_imgs = generator.predict(noise)

  # Train the discriminator
  # both loss and accuracy are returned
  d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
  d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
  d_loss = 0.5 * (d_loss_real + d_loss_fake)
  d_acc  = 0.5 * (d_acc_real + d_acc_fake)


  #######################
  ### Train generator ###
  #######################

  noise = np.random.randn(batch_size, latent_dim)
  g_loss = combined_model_gen.train_on_batch(noise, ones)

  # do it again!
  noise = np.random.randn(batch_size, latent_dim)
  g_loss = combined_model_gen.train_on_batch(noise, ones)

  # Save the losses
  d_losses.append(d_loss)
  g_losses.append(g_loss)

  if epoch % 100 == 0:
    print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, \
      d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")

  if epoch % sample_period == 0:
    sample_images(epoch)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


TypeError: unsupported format string passed to list.__format__

In [ ]:
# Main training loop
for epoch in range(epochs):
    ###########################
    ### Train discriminator ###
    ###########################

    # Select a random batch of images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]

    # Generate fake images
    noise = np.random.randn(batch_size, latent_dim)
    fake_imgs = generator.predict(noise)

    # Train the discriminator
    d_loss_real = discriminator.train_on_batch(real_imgs, ones)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, zeros)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    d_acc = np.mean(np.round(d_loss)) # Calculate discriminator accuracy

    #######################
    ### Train generator ###
    #######################

    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model_gen.train_on_batch(noise, ones)

    # Print progress for every epoch
    print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss[0]:.2f}, d_acc: {d_acc:.2f}, g_loss: {g_loss[0]:.2f}")

    if epoch % sample_period == 0:
        sample_images(epoch)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
epoch: 1/30000, d_loss: 2.76, d_acc: 1.50, g_loss: 2.76
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
epoch: 2/30000, d_loss: 2.76, d_acc: 1.50, g_loss: 2.76
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
epoch: 3/30000, d_loss: 2.76, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
epoch: 4/30000, d_loss: 2.77, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
epoch: 5/30000, d_loss: 2.77, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
epoch: 6/30000, d_loss: 2.77, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
epoch: 7/30000, d_loss: 2.77, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
epoch: 8/30000, d_loss: 2.77, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
epoch: 9/30000, d_loss: 2.77, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
epoch: 10/30000, d_loss: 2.77, d_acc: 1.50, g_loss: 2.77
1/1 ━━━━━━━━━━━━━━━━━